In [3]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

path='D:/B/CV(Open CV)/facial attendance' #path where the images are stored
images=[]
classNames=[]
myList=os.listdir(path)

for cl in myList:
    cur=cv2.imread(f'{path}/{cl}')
    images.append(cur)
    classNames.append(os.path.splitext(cl)[0])
 #encoding the faces   
def findEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
#writing attendance into the file
def markAttendance(name):
    with open('attendance.csv','r+') as f:
        myDataList=f.readlines()
        nameList=[]
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
            
        if name not in nameList:
            now=datetime.now()
            dtstring=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtstring}')
            
        
        

        
    
    


encodeprev=findEncodings(images)


cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame2=cv2.resize(frame,(0,0),None,0.25,0.25)
    frame2=cv2.cvtColor(frame2,cv2.COLOR_BGR2RGB)
    
    facesCurFrame=face_recognition.face_locations(frame2)
    encodeCurFrame=face_recognition.face_encodings(frame2,facesCurFrame)
    for encodeFace,faceloc in zip(encodeCurFrame, facesCurFrame):
        matches=face_recognition.compare_faces(encodeprev,encodeFace) #compares the faces
        faceDis=face_recognition.face_distance(encodeprev,encodeFace)  #distance btwn faces (the lesser difference the more similar)
        
        #print(faceDis)
        matchIndex=np.argmin(faceDis)#the face with min distance is selected as the recognised face
        
        if matches[matchIndex]:
            name=classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1=faceloc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),-1)
            cv2.putText(frame,name,(x1+6,y2-6),1,1,(255,255,255),2)
            markAttendance(name)        
        
    

    
    
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF==ord('w'):
        break
        
        
        
cap.release()
cv2.destroyAllWindows()
    
    

        

    